### 5 类型

In [2]:
# Julia 有类型系统
# 所有的值都有类型；但变量本身没有类型
# 你可以用 `typeof` 函数获得值的类型
typeof(5) # => Int64

Int64

In [3]:
# 类型是一等值
typeof(Int64)     # => DataType
typeof(DataType)  # => DataType
# DataType 是代表类型的类型，也代表他自己的类型

DataType

In [4]:
# 类型可用于文档化代码、执行优化以及多重派分(dispatch)
# Julia 并不只是静态的检查类型

# 用户还可以自定义类型
# 就跟其它语言的 record 或 struct 一样
# 用 `struct` 关键字定义新的类型

# struct Name
#   field::OptionalType
#   ...
# end
struct Tiger
    taillength::Float64
    coatcolor  # 不带类型标注相当于 `::Any`
end

# 默认构造函数的参数是类型的属性，按类型定义中的顺序排列
tigger = Tiger(3.5, "orange")  # => Tiger(3.5, "orange")

# 用新类型作为构造函数还会创建一个类型
sherekhan = typeof(tigger)(5.6, "fire")  # => Tiger(5.6, "fire")

# 类似 struct 的类型被称为具体类型
# 它们可被实例化，但不能有子类型

Tiger(5.6, "fire")

In [5]:
# 另一种类型是抽象类型

# 抽象类型名
abstract type Cat end  # 仅仅是指向类型结构层次的一个名称

# 抽象类型不能被实例化，但可以有子类型
# 例如，Number 就是抽象类型

subtypes(Number)
# => 2-element Vector{Any}:
# =>  Complex
# =>  Real

subtypes(Cat) # => Any[]

# AbstractString，类如其名，也是一个抽象类型
subtypes(AbstractString) 
# => 6-element Vector{Any}:
# => Core.Compiler.LazyString
# => LazyString
# => String
# => SubString
# => SubstitutionString
# => Test.GenericString

5-element Vector{Any}:
 Core.Compiler.LazyString
 LazyString
 String
 SubString
 SubstitutionString

In [6]:
# 所有的类型都有父类型。可以用函数 `supertype` 得到父类型
typeof(5) # => Int64
supertype(Int64)    # => Signed
supertype(Signed)   # => Integer
supertype(Integer)  # => Real
supertype(Real)     # => Number
supertype(Number)   # => Any
supertype(supertype(Signed))  # => Real
supertype(Any)      # => Any
# 除了 Int64 外，其余的类型都是抽象类型
typeof("fire")      # => String
supertype(String)   # => AbstractString
supertype(AbstractString) # => Any
supertype(SubString)  # => AbstractString


AbstractString

In [7]:
# <: 是子类型化操作符
struct Lion <: Cat  # Lion 是 Cat 的子类型
    mane_color
    roar::AbstractString
end

# 可以继续为你的类型定义构造函数
# 只需要定义一个与类型同名的函数，并调用已有的构造函数得到正确的类型
Lion(roar::AbstractString) = Lion("green", roar)  # => Lion
# 这是一个外部构造函数，因为它在类型定义之外

struct Panther <: Cat # Panther 也是 Cat 的子类型
    eye_color
    Panther() = new("green")
    # Panthers 只有这个构造函数，没有默认构造函数
end
# 像 Panther 一样使用内置构造函数，让你可以控制如何构建类型的值
# 应该尽量使用外部构造函数，而不是内部构造函

### 6 多分派（multiple distributed）

In [8]:
# Julia 中所有的函数都是通用函数，或者叫做泛型函数(generic functions)
# 也就是说这些函数都是由许多小方法组合而成的
# Lion 的每一种构造函数都是通用函数 Lion 的一个方法

# 我们来看一个非构造函数的例子
# 首先，让我们定义一个函数 meow

# Lion, Panther, Tiger 的 meow 定义分别为
function meow(animal::Lion)
    animal.roar
end
# => meow (generic function with 1 method)

function meow(animal::Panther)
    "grrr"
end
# => meow (generic function with 2 methods)

function meow(animal::Tiger)
    "rawwwr"
end
# => meow (generic function with 3 methods)

# 试试 meow 函数
meow(tigger)  # => "rawwwr"
meow(Lion("brown", "ROAAR"))  # => "ROAAR"
meow(Panther()) # => "grrr"

# 回顾类型的层次结构
Tiger <: Cat  # => false
Lion <: Cat  # => true
Panther <: Cat  # => true

true

In [9]:

function pet_cat(cat::Cat)
    println("The cat says $(meow(cat))")
end
# => pet_cat (generic function with 1 method)

pet_cat(Lion("42"))  # => The cat says 42
try
    pet_cat(tigger)  # => ERROR: MethodError: no method matching pet_cat(::Tiger)
catch e
    println(e)
end

The cat says 42


MethodError

(pet_cat, (Tiger(3.5, "orange"),), 0x0000000000007b08)


In [10]:
# 在面向对象语言中，通常都是单分派
# 这意味着使用的方法取决于第一个参数的类型
# 而 Julia 中选择方法时会考虑到所有参数的类型

# 让我们定义一个有更多参数的函数，这样我们就能看出区别
function fight(t::Tiger, c::Cat)
    println("The $(t.coatcolor) tiger wins!")
end
# => fight (generic function with 1 method)

fight(tigger, Panther())  # => The orange tiger wins!
fight(tigger, Lion("ROAR")) # => fight(tigger, Lion("ROAR"))


The orange tiger wins!
The orange tiger wins!


In [11]:
# 让我们修改一下传入 Lion 类型时的行为
fight(t::Tiger, l::Lion) = println("The $(l.mane_color)-maned lion wins!")
# => fight (generic function with 2 methods)

fight(tigger, Panther())  # => The orange tiger wins!
fight(tigger, Lion("ROAR"))  # => The brown-maned lion wins!

The orange tiger wins!
The green-maned lion wins!


In [12]:
# 我们不需要一只老虎参与战斗
fight(l::Lion, c::Cat) = println("The victorious cat says $(meow(c))")
# => fight (generic function with 3 methods)

fight(Lion("balooga!"), Panther())  # => The victorious cat says grrr
try
    fight(Panther(), Lion("RAWR"))
    # => ERROR: MethodError: no method matching fight(::Panther, ::Lion)
    # => Closest candidates are:
    # =>   fight(::Tiger, ::Lion) at ...
    # =>   fight(::Tiger, ::Cat) at ...
    # =>   fight(::Lion, ::Cat) at ...
    # => ...
catch e
    println(e)
end

The victorious cat says grrr
MethodError(fight, (Panther("green"), Lion("green", "RAWR")), 0x0000000000007b0b)


In [13]:
# 试试把 Cat 放在前面
fight(c::Cat, l::Lion) = println("The cat beats the Lion")
# => fight (generic function with 4 methods)

# 由于无法判断该使用哪个 fight 方法，而产生了错误
try
    fight(Lion("RAR"), Lion("brown", "rarrr"))
    # => ERROR: MethodError: fight(::Lion, ::Lion) is ambiguous. Candidates:
    # =>   fight(c::Cat, l::Lion) in Main at ...
    # =>   fight(l::Lion, c::Cat) in Main at ...
    # => Possible fix, define
    # =>   fight(::Lion, ::Lion)
    # => ...
catch e
    println(e)
end

MethodError(fight, (Lion("green", "RAR"), Lion("brown", "rarrr")), 0x0000000000007b0c)


In [14]:
fight(l::Lion, l2::Lion) = println("The lions come to a tie")
fight(Lion("RAR"), Lion("brown", "rarrr"))

The lions come to a tie
